## Chapter 21. 클래스 메타프로그래밍
클래스 메타프로그래밍은 실행 도중에 클래스를 생성하거나 커스터마이즈하는 기술을 말한다. 클래스는 파이썬의 일급 객체이므로 class라는 키워드를 사용하지 않고도 언제든 함수를 사용해서 생성할 수 있다. 클래스 데커레이터도 함수지만 클래스를 조사, 변경하거나 심지어 다른 클래스로 대체할 수 있다. 마지막으로 메타클래스는 클래스 메타프로그래밍을 하기 위한 최첨단 도구이며 우리가 이미 살펴본 추상 베이스 클래스처럼 특별한 기질이 있는 완전히 새로운 부류의 클래스를 만들 수 있게 해준다.

```
CAUTION_: 메타 프로그래밍은 흥미로운 주제이나 프레임워크를 만들고 있지 않다면 메타 클래스를 작성해서는 안 된다. 재미로 하거나 배운 개념을 적용해보기 위한 것이 아니라면...
```

### 21.1 클래스 팩토리
collectoins.namedtuple()에서 힌트를 얻어 Dog 같은 간단한 클래스를 즉석으로 생성하는 record_factory()를 만들어보자. 

In [15]:
""" [예제 21-2] factories.py: 간단한 클래스 팩토리 """

def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: # replace()나 split()을 사용할 수 없다.
        pass
    
    field_names = tuple(field_names) # 속성명들의 튜플을 생성한다. 이 투플은 새로운 클래스의 __slots__에 들어 있는 속성으로 사용하며,
                                     # 언패킹과 __repr__()에 사용할 필드 순서도 설정한다.
    
    def __init__(self, *args, **kwargs): # 새로운 클래스의 __init__() 메서드가 된다. 위치 인수나 키워드 인수를 받는다.
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self): # __slot__ 와 self를 반복해서 출력하는 __repr__() 메서드를 정의한다.
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return "{}({})".format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names, # 클래스 속성의 딕셔너리를 조합한다. 
                     __init__ = __init__,
                     __iter__ = __iter__,
                     __repr__ = __repr__)
    
    return type(cls_name, (object,), cls_attrs) # type() 생성자를 호출해서 새로운 클래스를 생성하고 반환한다.

In [16]:
 Dog = record_factory('Dog', 'name weight owner')

In [17]:
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [18]:
name, weight, _ = rex
name, weight

('Rex', 30)

In [19]:
"{2}'s dog weighs 30kg'".format(*rex)

"Bob's dog weighs 30kg'"

In [20]:
rex.weight = 32 # 팩토리 객체는 가변형
rex

Dog(name='Rex', weight=32, owner='Bob')

In [21]:
Dog.__mro__ # 새로 생성된 클래스는 object를 상속하며, 팩토리와는 아무 관련이 없다.

(__main__.Dog, object)

type(my_object)를 이용해서 객체의 클래스와 동일한 my_object.\_\_class\_\_를 가져오므로 type()을 함수로 생각하기 쉬우나 type은 클래스이다. 다음과 같이 인수 세 개를 받아서 호출하면 새로운 클래스를 생성하는 일종의 클래스처럼 작동한다.
```
Myclass = type('MyClass', (MySuperClass, MyMixin), {'x':42, 'x2': lamba self: self.x*2}) 
```
위 코드는 기능상으로 다음 코드와 동일하다.

```
class MyClass(MySuperClass, MyMixin):
    x = 42
    
    def x2(self):
        return self.x * 2
```

특이한 점은 여기에 나온 MyClass 나 [예제 21-1]의 Dog처럼 type의 객체가 클래스라는 것이다.

record_factory()로 생성한 클래스의 객체들은 직렬화할 수 없다는 제한이 있다. 즉, pickle 모듈의 dump(), load() 함수와 함께 사용할 수 없다. 이 절의 목표는 간단한 사례에서 type 클래스의 사용법을 보여주는 것이므로, 이 문제를 해결하는 것은 이 예제의 범위를 벗어난다. 

### 21.2 디스크립터를 커스터마이즈하기 위한 클래스 데커레이터
저장소 명이 '_Quantity#0'과 같았던 Chapter 20의 문제를 실제 관리 대상 속성의 이름을 포함하도록 수정하였다.

In [28]:
""" [예제 20-6]와 동일 """
import abc

class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value) # 검증 기능은 상속받은 클래스에 구현되어 있으며 상위 클래스 AutoStroage는 저장만 담당한다.

        
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value): 
        value = self.validate(instance, value) # validate 함수에 검증을 위임한다. 
        super().__set__(instance, value) # 검증 후 반환된 값을 이용해서 실제로 값을 저장하는 슈퍼클래스의 __set__() 메서드를 호출한다.
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """ 검증된 값을 반환하거나 ValueError를 발생시킨다. """

        
class Quantity(Validated):
    """ 양수임을 검증 """
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("value must be > 0")
        else:
            return value # 구상 메서드가 검증된 값을 반환하도록 구현하여 전달받은 데이터를 정리, 변환
        
        
class NonBlank(Validated):
    """ 최소 하나 이상의 비공백 문자가 들어있는 문자열 """
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value
    
def entity(cls):
    for key, attr in cls.__dict__.items():
        if isinstance(attr, Validated): # 속성이 Validated 디스크립터 클래스인지 확인한다.
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)
            
    return cls

In [29]:
""" [예제 21-3] bulkfood_v6.py: Quanity와 NonBlank 디스크립터를 사용하는 LineItem 클래스 """
@entity
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [30]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3] # # dir(raisins) means that "prints list which contains names of attributes in 'raisins'"

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [31]:
LineItem.description.storage_name

'_NonBlank#description'

In [32]:
raisins.description

'Golden raisins'

In [33]:
getattr(raisins, '_NonBlank#description')

'Golden raisins'

클래스 데커레이터는 자신에게 직접 적용된 클래스에서만 작동할 수 있다는 큰 단점이 있다. 즉, 장식된 클래스의 변경된 내용에 따라 서브클래스가 변경된 내용을 상속할 수 도 아닐 수도 있으므로 이문제의 해결책을 알아본다.

### 21.3 임포트 타임과 런타임
#### 21.3.1 코드 평가시점 연습 문제
